# Homework 2 - Data Wrangling with Hadoop

The goal of this assignment is to put into action the data wrangling techniques from the exercises of week-3 and week-4. We highly suggest you to finish these two exercises first and then start the homework. In this homework, we are going to reuse the same __sbb__ and __twitter__ datasets as seen before in these two exercises. 

## Hand-in Instructions
- __Due: 13.04.2021 23:59 CET__
- `git push` your final verion to your group's Renku repository before the due date
- Verify that `Dockerfile`, `environment.yml` and `requirements.txt` are properly written and notebook is functional
- Add necessary comments and discussion to make your queries readable

## Hive Documentation

Hive queries: <https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select>

Hive functions: <https://cwiki.apache.org/confluence/display/Hive/LanguageManual+UDF>

<div style="font-size: 150%" class="alert alert-block alert-warning">
    <b>Get yourself ready:</b> 
    <br>
    Before you jump into the questions, please first go through the notebook <a href='./prepare_env.ipynb'>prepare_env.ipynb</a> and make sure that your environment is properly set up.
    <br><br>
    <b>Cluster Usage:</b>
    <br>
    As there are many of you working with the cluster, we encourage you to prototype your queries on small data samples before running them on whole datasets.
    <br><br>
    <b>Try to use as much HiveQL as possible and avoid using pandas operations.</b>
</div>

## Part I: SBB/CFF/FFS Data (40 Points)

Data source: <https://opentransportdata.swiss/en/dataset/istdaten>

In this part, you will leverage Hive to perform exploratory analysis of data published by the [Open Data Platform Swiss Public Transport](https://opentransportdata.swiss).

Format: the dataset is originally presented as a collection of textfiles with fields separated by ';' (semi-colon). For efficiency, the textfiles have been compressed into Optimized Row Columnar ([ORC](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+ORC)) file format. 

Location: you can find the data in ORC format on HDFS at the path `/data/sbb/orc/istdaten`.

The full description from opentransportdata.swiss can be found in <https://opentransportdata.swiss/de/cookbook/ist-daten/> in four languages. Because of the translation typos there may be some misunderstandings. We suggest you rely on the German version and use an automated translator when necessary. We will clarify if there is still anything unclear in class and Slack. Here are the relevant column descriptions:

- `BETRIEBSTAG`: date of the trip
- `FAHRT_BEZEICHNER`: identifies the trip
- `BETREIBER_ABK`, `BETREIBER_NAME`: operator (name will contain the full name, e.g. Schweizerische Bundesbahnen for SBB)
- `PRODUKT_ID`: type of transport, e.g. train, bus
- `LINIEN_ID`: for trains, this is the train number
- `LINIEN_TEXT`,`VERKEHRSMITTEL_TEXT`: for trains, the service type (IC, IR, RE, etc.)
- `ZUSATZFAHRT_TF`: boolean, true if this is an additional trip (not part of the regular schedule)
- `FAELLT_AUS_TF`: boolean, true if this trip failed (cancelled or not completed)
- `HALTESTELLEN_NAME`: name of the stop
- `ANKUNFTSZEIT`: arrival time at the stop according to schedule
- `AN_PROGNOSE`: actual arrival time
- `AN_PROGNOSE_STATUS`: show how the actual arrival time is calcluated
- `ABFAHRTSZEIT`: departure time at the stop according to schedule
- `AB_PROGNOSE`: actual departure time
- `AB_PROGNOSE_STATUS`: show how the actual departure time is calcluated
- `DURCHFAHRT_TF`: boolean, true if the transport does not stop there

Each line of the file represents a stop and contains arrival and departure times. When the stop is the start or end of a journey, the corresponding columns will be empty (`ANKUNFTSZEIT`/`ABFAHRTSZEIT`).

In some cases, the actual times were not measured so the `AN_PROGNOSE_STATUS`/`AB_PROGNOSE_STATUS` will be empty or set to `PROGNOSE` and `AN_PROGNOSE`/`AB_PROGNOSE` will be empty.

__Initialization__

In [ ]:
import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go

username = os.environ['JUPYTERHUB_USER']
hiveaddr = os.environ['HIVE_SERVER_2']
print("Operating as: {0}".format(username))

In [ ]:
from pyhive import hive

# create connection
conn = hive.connect(host=hiveaddr, 
                    port=10000,
                    username=username) 
# create cursor
cur = conn.cursor()

### a) Type of transport - 10/40

In the exercise of week-3, you have already explored the stop distribution of different types of transport on 05.11.2018. Now, let's do the same for the whole dataset.

- Query `sbb_orc` to get the total number of stops for different types of transport in each month, and order it by time and type of transport.
|month_year|ttype|stops|
|---|---|---|
|...|...|...|
- Use `plotly` to create a facet bar chart partitioned by the type of transportation. 
- Document any patterns or abnormalities you can find.

__Note__: 
- In general, one entry in the `sbb_orc` table means one stop.
- You might need to filter out the rows where:
    - `BETRIEBSTAG` is not in the format of `__.__.____`
    - `PRODUKT_ID` is NULL or empty
- Facet plot with plotly: https://plotly.com/python/facet-plots/

In [ ]:
query = """
    select from_unixtime(unix_timestamp(BETRIEBSTAG, 'dd.MM.yyyy'), 'yyyy-MM') as month_year, 
        lower(PRODUKT_ID) as ttype,
        count(*) as stops
    from {0}.sbb_orc
    where BETRIEBSTAG != '__.__.____' and PRODUKT_ID is not null and length(PRODUKT_ID) > 0 and length(BETRIEBSTAG)>0 and date_format(to_date(from_unixtime(unix_timestamp(BETRIEBSTAG, 'dd.MM.yyyy'))), 'yyyy-MM') is not null
    group by lower(PRODUKT_ID), from_unixtime(unix_timestamp(BETRIEBSTAG, 'dd.MM.yyyy'), 'yyyy-MM')
    order by month_year asc
""".format(username)
df_ttype = pd.read_sql(query, conn)

In [ ]:
# Show the table
df_ttype

In [ ]:
fig = px.bar(
    df_ttype,
    x="month_year",
    y="stops",
    color="ttype",
    labels={"month_year": "Months",
            "stops": "Number of stops",
            "ttype": "Types of transport"
           },
    title="The stop distribution of different types of transports over different months"
)

fig.show()

**Documents:** According to the table, we find that bus, train(zug), tram are the most commonly used types of transportation. For each type of transportaion, the number of stops of bus is increasing year by year from 2018 to 2020; the usage of schiff reaches its peak at every August; the use of zahnradbahn is on the rise and is slowly surpassing the use of metros from 2020. Otherwise, we find the stops in July 2019 is significantly less than other months for all the types of transportation except for ahnradbahn.

### b) Schedule - 10/40

- Select a typical week day (not Saturday, not Sunday, not a bank holiday) from `sbb_orc`. Query the one-day table and get the set of IC (`VERKEHRSMITTEL_TEXT`) trains you can take to go (without connections) from Genève to Lausanne on that day. 
- Display the train number (`LINIEN_ID`) as well as the schedule (arrival and departure time) of the trains.

|train_number|departure|arrival|
|---|---|---|
|...|...|...|

__Note:__ 
- The schedule of IC from Genève to Lausanne has not changed for the past few years. You can use the advanced search of SBB's website to check your answer.
- Do not hesitate to create intermediary tables. 
- You might need to add filters on these flags: `ZUSATZFAHRT_TF`, `FAELLT_AUS_TF`, `DURCHFAHRT_TF` 
- Functions that could be useful: `unix_timestamp`, `to_utc_timestamp`, `date_format`.

In [ ]:
# Drop table if exists
query = """
    drop table if exists {0}.geneve_station
""".format(username)
cur.execute(query)

# Create a table for IC trains in Geneva for a week day (e.g. 10/10/2019)
query = """
    create table {0}.geneve_station
    stored as orc
    as 
        select *
        from {0}.sbb_orc
        where VERKEHRSMITTEL_TEXT like 'IC' 
        and HALTESTELLEN_NAME like 'Genève' 
        and BETRIEBSTAG like '10.10.2019'
        and FAELLT_AUS_TF like 'false'
        and DURCHFAHRT_TF like 'false'
""".format(username)
cur.execute(query)

# Drop table if exists
query = """
    drop table if exists {0}.lausanne_station
""".format(username)
cur.execute(query)

# Create a table for IC trains in Lausanne for a week day (e.g. 10/10/2019)
query = """
    create table {0}.lausanne_station
    stored as orc
    as 
        select *
        from {0}.sbb_orc
        where VERKEHRSMITTEL_TEXT like 'IC' 
        and HALTESTELLEN_NAME like 'Lausanne' 
        and BETRIEBSTAG like '10.10.2019'
        and FAELLT_AUS_TF like 'false'
        and DURCHFAHRT_TF like 'false'
""".format(username)
cur.execute(query)

In [ ]:
query = """
    select  
        geneve_station.LINIEN_ID as train_number, 
        to_utc_timestamp(1000*unix_timestamp(geneve_station.AB_PROGNOSE, 'dd.MM.yyy HH:mm'),"UTC") as departure,
        to_utc_timestamp(1000*unix_timestamp(lausanne_station.AN_PROGNOSE, 'dd.MM.yyy HH:mm'),"UTC") as arrival  
    from {0}.geneve_station FULL OUTER JOIN {0}.lausanne_station ON (geneve_station.LINIEN_ID = lausanne_station.LINIEN_ID)
    where 
        unix_timestamp(lausanne_station.AN_PROGNOSE, 'dd.MM.yyy HH:mm') > 0 and 
        unix_timestamp(geneve_station.AB_PROGNOSE, 'dd.MM.yyy HH:mm') > 0 and 
        unix_timestamp(lausanne_station.AN_PROGNOSE, 'dd.MM.yyy HH:mm') > unix_timestamp(geneve_station.AB_PROGNOSE, 'dd.MM.yyy HH:mm')
    order by geneve_station.LINIEN_ID
""".format(username)
df_geneve_lausanne = pd.read_sql(query, conn)

In [ ]:
# Show the table
df_geneve_lausanne

### c) Delay percentiles - 10/40

- Query `sbb_orc` to compute the 50th and 75th percentiles of __arrival__ delays for IC 702, 704, ..., 728, 730 (15 trains total) at Genève main station. 
- Use `plotly` to plot your results in a proper way. 
- Which trains are the most disrupted? Can you find the tendency and interpret?

__Note:__
- Do not hesitate to create intermediary tables. 
- When the train is ahead of schedule, count this as a delay of 0.
- Use only stops with `AN_PROGNOSE_STATUS` equal to __REAL__ or __GESCHAETZT__.
- Functions that may be useful: `unix_timestamp`, `percentile_approx`, `if`

In [ ]:
# Drop table if exists
query = """
    drop table if exists {0}.geneva_main_station
""".format(username)
cur.execute(query)

# Select trains
query = """
    create table {0}.geneva_main_station
    stored as orc
    as 
        select LINIEN_ID as id,
               unix_timestamp(ANKUNFTSZEIT, 'dd.MM.yyyy HH:mm') as expected_arrival, 
               unix_timestamp(AN_PROGNOSE, 'dd.MM.yyyy HH:mm:ss') as actual_arrival
        from {0}.sbb_orc
        where 
            VERKEHRSMITTEL_TEXT like 'IC' and 
            HALTESTELLEN_NAME like 'Genève' and 
            (AN_PROGNOSE_STATUS like 'REAL' or AN_PROGNOSE_STATUS like 'GESCHAETZT') and 
            cast(LINIEN_ID as int) >= 702 and 
            cast(LINIEN_ID as int) <= 730 and 
            (cast(LINIEN_ID as int)% 2) == 0
""".format(username)
cur.execute(query)

In [ ]:
# Give the delay of each entry
query = """
   select id, 
        if (actual_arrival > expected_arrival, actual_arrival - expected_arrival, 0) as delay
    from {0}.geneva_main_station
    order by delay asc
""".format(username)
df_delays_ic_gen=pd.read_sql(query, conn)

In [ ]:
# Compute the 50th and 75th percentiles of arrival delays for different IC trains
query="""
with delay_IC as(
    select id, 
           from_unixtime(expected_arrival, 'HH:mm') as schedule_arrival,
           if (actual_arrival > expected_arrival, actual_arrival - expected_arrival, 0) as delay
    from {0}.geneva_main_station
    order by delay asc
)
select id,
       percentile_approx(delay,0.5) as 50th,
       percentile_approx(delay,0.75) as 75th,
       schedule_arrival
       from delay_IC
       group by id, schedule_arrival
       order by id
""".format(username)
df_delays_ic_gen_each = pd.read_sql(query, conn)

In [ ]:
# Show the table
df_delays_ic_gen_each

In [ ]:
print('50th percentile of delays (in seconds) over all trains:')
df_delays_ic_gen['delay'].quantile(0.5)

In [ ]:
print('75th percentile of delays (in seconds) over all trains:')
df_delays_ic_gen['delay'].quantile(0.75)

In [ ]:
df_delays_ic_gen_each=pd.melt(df_delays_ic_gen_each,id_vars=['id', 'schedule_arrival'],var_name='percentile', value_name='value')
fig = px.bar(
    df_delays_ic_gen_each,
    x="id",
    y='value',
    color='percentile',
    barmode='group',
    labels={"id": "Train number",
            "percentile": "Percentile",
            "value": "Delays (in seconds)"
           },
    title="The delay distribution of different train numbers",
)

fig.update_xaxes(type='category', categoryorder='category ascending')
fig.show()

**Solution**: According to 75th percentiles, train 708 is most disrupted. According to 50th percentiles, train 726 is most disrupted. Therefore, train 708 and 726 are most disrupted. We find that trains run during morning and evening peak are more prone to delay.

### d) Delay heatmap 10/40

- For each week (1 to 52) of each year (2018 to 2020), query `sbb_orc` to compute the median of delays of all trains __departing__ from any train stations in Zürich area during that week. 
- Use `plotly` to draw a heatmap (year x week) of the median delays. 
- In which weeks were the trains delayed the most/least? Can you explain the results?

__Note:__
- Do not hesitate to create intermediary tables. 
- When the train is ahead of schedule, count this as a delay of 0.
- Use only stops with `AB_PROGNOSE_STATUS` equal to __REAL__ or __GESCHAETZT__.
- For simplicty, a train station in Zürich area <=> it's a train station & its `HALTESTELLEN_NAME` starts with __Zürich__.
- Heatmap with `plotly`: https://plotly.com/python/heatmaps/
- Functions that may be useful: `unix_timestamp`, `from_unixtime`, `weekofyear`, `percentile_approx`, `if`

In [ ]:
# Drop table if exists
query = """
    drop table if exists {0}.zurich_station
""".format(username)
cur.execute(query)

query = """
    create table {0}.zurich_station
    stored as orc
    as 
        select LINIEN_ID as id,
               weekofyear(TO_DATE(FROM_UNIXTIME(unix_timestamp(BETRIEBSTAG, 'dd.MM.yyyy')))) as week,
               Year(TO_DATE(FROM_UNIXTIME(unix_timestamp(BETRIEBSTAG, 'dd.MM.yyyy')))) as year,
               unix_timestamp(ABFAHRTSZEIT, 'dd.MM.yyyy HH:mm') as expected_departure, 
               unix_timestamp(AB_PROGNOSE, 'dd.MM.yyyy HH:mm:ss') as actual_departure
        from {0}.sbb_orc
        where HALTESTELLEN_NAME like 'Zürich%' 
        and PRODUKT_ID like 'Zug'
        and (AB_PROGNOSE_STATUS like 'REAL' or AB_PROGNOSE_STATUS like 'GESCHAETZT')
        
""".format(username)
cur.execute(query)

In [ ]:
query = """
with delay_zurich as(
select id, week, year,
        case
            when actual_departure > expected_departure then actual_departure - expected_departure
            else 0
        end as delay
    from {0}.zurich_station
    order by delay asc
)
select week, year,percentile_approx(delay,0.5) as medians
from delay_zurich
group by week, year
order by year, week
""".format(username)
df_delays_zurich = pd.read_sql(query, conn)

In [ ]:
import numpy as np
df_delays_zurich = df_delays_zurich.set_index(['year', 'week'])
# Fill up missed data
add_indices = pd.Index((i, j) for i in range(2018, 2021) for j in range(1, 53)).difference(df_delays_zurich.index)
add_df = pd.DataFrame(index=add_indices, columns=df_delays_zurich.columns).fillna(np.NaN)
df_delays_zurich = pd.concat([df_delays_zurich, add_df])
# Delete 53th week in 2020 to keep 52 weeks each year
df_delays_zurich = df_delays_zurich.drop((2020, 53))
df_delays_zurich = df_delays_zurich.loc[[(i, j) for i in range(2018, 2021) for j in range(1, 53)], :]

In [ ]:
# Show the table
df_delays_zurich

In [ ]:
fig = px.imshow(
    df_delays_zurich.to_numpy().reshape((3, -1)),
    x = [str(i) for i in range (1, 53)],
    y = [str(i) for i in range(2018, 2021)],
    aspect = 'auto',
    labels=dict(x="Week", y="Year", color="Delays (in second)"),
    title='Median of delays (in second) of all trains for different week',
)
fig.show()

**Solution**: According to the heatmap, in the 44th week of 2019 (Oct. 28 to Nov. 3), the trains delayed the most. Reasons might be: 1)Federal elections were held in Switzerland on 20 October 2019 to elect all members of both houses of the Federal Assembly; 2) Daylight saving time ends in late October 2019.  
In the 14th week of 2020 (Mar. 30 to Apr. 5), the trains delayed the least. Reasons might be: 1) Because of the Covid situation, people started to work remotely.

## Part II: Twitter Data (20 Points)

Data source: https://archive.org/details/twitterstream?sort=-publicdate 

In this part, you will leverage Hive to extract the hashtags from the source data, and then perform light exploration of the prepared data. 

### Dataset Description 

Format: the dataset is presented as a collection of textfiles containing one JSON document per line. The data is organized in a hierarchy of folders, with one file per minute. The textfiles have been compressed using bzip2. In this part, we will mainly focus on __2020 twitter data__.

Location: you can find the data on HDFS at the path `/data/twitter/json/2020/{month}/{day}/{hour}/{minute}.json.bz2`. 

Relevant fields: 
- `created_at`, `timestamp_ms`: The first is a human-readable string representation of when the tweet was posted. The latter represents the same instant as a timestamp in seconds since UNIX epoch. 
- `lang`: the language of the tweet content 
- `entities`: parsed entities from the tweet, e.g. hashtags, user mentions, URLs.
- In this repository, you can find [a tweet example](../data/tweet-example.json).

<div style="font-size: 100%" class="alert alert-block alert-danger">
    <b>Disclaimer</b>
    <br>
    This dataset contains unfiltered data from Twitter. As such, you may be exposed to tweets/hashtags containing vulgarities, references to sexual acts, drug usage, etc.
    </div>

### a) JsonSerDe - 4/20

In the exercise of week 4, you have already seen how to use the [SerDe framework](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-RowFormats&SerDe) to extract JSON fields from raw text format. 

In this question, please use SerDe to create an <font color="red" size="3px">EXTERNAL</font> table with __one day__ twitter data. You only need to extract three columns: `timestamp_ms`, `lang` and `entities`(with the field `hashtags` only) with following schema (you need to figure out what to fill in `...`):
```
timestamp_ms string,
lang         string,
entities     struct<hashtags:array<...<text:..., indices:...>>>
```

The table you create should be similar to:

| timestamp_ms | lang | entities |
|---|---|---|
| 1234567890001 | en | {"hashtags":[]} |
| 1234567890002 | fr | {"hashtags":[{"text":"hashtag1","indices":[10]}]} |
| 1234567890002 | jp | {"hashtags":[{"text":"hashtag1","indices":[14,23]}, {"text":"hashtag2","indices":[45]}]} |

__Note:__
   - JsonSerDe: https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-RowFormats&SerDe
   - Hive data types: https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Types#LanguageManualTypes

In [ ]:
# Drop table if exists
query="""
    drop table if exists {0}.hashtags_one_day_intermediary
""".format(username)
cur.execute(query)

# Create a external table with one day twitter data (e.g. 10/10/2020)
query="""
    create external table {0}.hashtags_one_day_intermediary(    
        timestamp_ms string,
        lang string,
        entities struct<hashtags:array<struct<text:string, indices:array<int>>>>
    )
    row format serde 'org.apache.hadoop.hive.serde2.JsonSerDe'
    stored as textfile
    location '/data/twitter/json/2020/10/10'
""".format(username)
cur.execute(query)

# Show the table
query="""
    select * from {0}.hashtags_one_day_intermediary limit 10
""".format(username)
pd.read_sql(query, conn)

### b) Explosion - 4/20

In a), you create a table where each row could contain a list of multiple hashtags. Create another table by normalizing the table obtained from the previous step. This means that each row should contain exactly one hashtag. Include `timestamp_ms` and `lang` in the resulting table, as shown below.

| timestamp_ms | lang | hashtag |
|---|---|---|
| 1234567890001 | es | hashtag1 |
| 1234567890001 | es | hashtag2 |
| 1234567890002 | en | hashtag2 |
| 1234567890003 | zh | hashtag3 |

__Note:__
   - `LateralView`: https://cwiki.apache.org/confluence/display/Hive/LanguageManual+LateralView
   - `explode` function: <https://cwiki.apache.org/confluence/display/Hive/LanguageManual+UDF#LanguageManualUDF-explode>

In [ ]:
# Drop table if exists
query="""
    drop table if exists {0}.hashtags_one_day
""".format(username)
cur.execute(query)

# Create normalized table
query="""
    create table if not exists {0}.hashtags_one_day
    stored as orc
    as
        select timestamp_ms, lang, hashtag
        from {0}.hashtags_one_day_intermediary lateral view explode(entities.hashtags.text) adTable as hashtag
        
""".format(username)
cur.execute(query)

# Show the table
query="""
    select * from {0}.hashtags_one_day limit 10
""".format(username)
pd.read_sql(query, conn)

### c) Hashtags - 8/20

Query the normailized table you obtained in b). Create a table of the top 20 most mentioned hashtags with the contribution of each language. And, for each hashtag, order languages by their contributions. You should have a table similar to:

|hashtag|lang|lang_count|total_count|
|---|---|---|---|
|hashtag_1|en|2000|3500|
|hashtag_1|fr|1000|3500|
|hashtag_1|jp|500|3500|
|hashtag_2|te|500|500|

Use `plotly` to create a stacked bar chart to show the results.

__Note:__ to properly order the bars, you may need:
```python
fig.update_layout(xaxis_categoryorder = 'total descending')
```

In [ ]:
# Drop table if exists
query="""
    drop table if exists {0}.count_by_lang
""".format(username)
cur.execute(query)

# Create table to count by language
query="""
    create table if not exists {0}.count_by_lang
    stored as orc
    as
        select hashtag, lang, count(*) as lang_count
        from {0}.hashtags_one_day
        group by hashtag, lang
""".format(username)
cur.execute(query)

In [ ]:
# Drop table if exists
query="""
    drop table if exists {0}.count_by_hashtag
""".format(username)
cur.execute(query)

# Create table to count by hashtag
query="""
    create table if not exists {0}.count_by_hashtag
    stored as orc
    as
        select hashtag, lang, lang_count, 
        sum(lang_count) over (partition by hashtag) as total_count
        from {0}.count_by_lang
""".format(username)
cur.execute(query)

In [ ]:
# Drop table if exists
query="""
    drop table if exists {0}.top_hashtags_one_day
""".format(username)
cur.execute(query)

# Create table for ranking
query="""
    create table if not exists {0}.top_hashtags_one_day
    stored as orc
    as
        select *,
        dense_rank() over(order by total_count desc) as rank
        from {0}.count_by_hashtag
""".format(username)
cur.execute(query)

In [ ]:
# Drop table if exists
query="""
    drop table if exists {0}.top20_hashtags_one_day
""".format(username)
cur.execute(query)

# Create table to select top 20
query="""
    create table if not exists {0}.top20_hashtags_one_day
    stored as orc
    as 
        select hashtag, lang, lang_count, total_count
        from {0}.top_hashtags_one_day 
        where rank<=20
        sort by total_count desc, lang_count desc
""".format(username)
cur.execute(query)

query="""
    select * from {0}.top20_hashtags_one_day 
""".format(username)
df_hashtag = pd.read_sql(query, conn)

In [ ]:
# Show the table
df_hashtag

In [ ]:
fig = px.bar(
    df_hashtag,
    x='top20_hashtags_one_day.hashtag',
    y='top20_hashtags_one_day.lang_count',
    color='top20_hashtags_one_day.lang',
    labels={'top20_hashtags_one_day.hashtag':'Hashtag',
            'top20_hashtags_one_day.lang_count':'Number of tweets',
            'top20_hashtags_one_day.lang':'Language'
            },
    title='Top 20 most mentioned hashtags with the contribution of each language'
)

fig.update_layout(xaxis_categoryorder = 'total descending')
fig.show()


### d) HBase - 4/20

In the lecture and exercise of week-4, you have learnt what's HBase, how to create an Hbase table and how to create an external Hive table on top of the HBase table. Now, let's try to save the results of question c) into HBase, such that each entry looks like:
```
(b'PIE', {b'cf1:total_count': b'31415926', b'cf2:langs': b'ja,en,ko,fr'})
``` 
where the key is the hashtag, `total_count` is the total count of the hashtag, and `langs` is a string of language abbreviations concatenated with commas. 

__Note:__
- To accomplish the task, you need to follow these steps:
    - Create an Hbase table called `twitter_hbase`, in **your hbase namespace**, with two column families and fields (cf1, cf2)
    - Create an external Hive table called `twitter_hive_on_hbase` on top of the Hbase table. 
    - Populate the HBase table with the results of question c).
- You may find function `concat_ws` and `collect_list` useful.

In [ ]:
import happybase
hbaseaddr = os.environ['HBASE_SERVER']
hbase_connection = happybase.Connection(hbaseaddr, transport='framed',protocol='compact')

In [ ]:
try:
    hbase_connection.delete_table('{0}:twitter_hbase'.format(username),disable=True)
except Exception as e:
    pass

hbase_connection.create_table(
    '{0}:twitter_hbase'.format(username),
    {'cf1': dict(),
     'cf2': dict()
    }
)

In [ ]:
# Drop table if exists
query = """
drop table {0}.twitter_hive_on_hbase
""".format(username)
cur.execute(query)

query = """
create external table {0}.twitter_hive_on_hbase(
    RowKey string,
    total_count bigint,
    langs string
)
STORED BY 'org.apache.hadoop.hive.hbase.HBaseStorageHandler'
WITH SERDEPROPERTIES (
    "hbase.columns.mapping"=":key,cf1:total_count,cf2:langs"
)
TBLPROPERTIES(
    "hbase.table.name"="{0}:twitter_hbase",
    "hbase.mapred.output.outputtable"="{0}:twitter_hbase"
)
""".format(username)
cur.execute(query)

query="""
insert overwrite table {0}.twitter_hive_on_hbase
    select
         hashtag as RowKey,
         total_count as total_count,
         concat_ws(',',collect_set(cast(lang as string))) as langs
    from {0}.top20_hashtags_one_day
    group by total_count, hashtag
""".format(username)
cur.execute(query)

In [ ]:
# Show the table
for i, r in enumerate(hbase_connection.table('{0}:twitter_hbase'.format(username)).scan()):
    print(r)

# That's all, folks!